In [1]:
#!pip install google-cloud-bigquery pandas db-dtypes google-cloud-bigquery-storage
import os
from google.cloud import bigquery
import pandas as pd

import db_dtypes
print("db-dtypes is installed correctly.")


# Set the credentials for Google BigQuery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "st312-442610-b9934d6c7287.json"
client = bigquery.Client()


db-dtypes is installed correctly.


### test Query - this is a small test query to see if everything is working

In [2]:
query = """
SELECT distinct to_address FROM `bigquery-public-data.crypto_polygon.token_transfers`
WHERE from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"
AND block_timestamp BETWEEN "2024-08-01 00:00:00" AND "2024-08-31 23:59:59"
LIMIT 1000;
"""


# Run query and load into Pandas DataFrame
df = client.query(query).to_dataframe()

# Display results
df

,to_address
0,0xd36ec33c8bed5a9f7b6630855f1533455b98a418
1,0x4697bb208116e266a7a60a7d9f7e9d81df7293c6
2,0xc5d563a36ae78145c45a50134d48a1215220f80a
3,0x4bfb41d5b3570defd03c39a9a4d8de6bd8b8982e


### Game plan
1. Find the month we want - we doing 8th of 2024
2. Get all the contracts big people interact with on poly market

2. Get all info from those contracts

--- Contracts



-- Goal to indeitfy polymarket contracts



-- tried  address for

-- 0x39Ba731cFA2828ea64787AE165FD1e78Fe43AEDe and 0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E 

-- hit on 0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74!

 --SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.contracts ⁠ where address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74" LIMIT 1000





--- Now I used that to attak the logs



-- I got loads of data but only about AncillaryDataUpdated(bytes32,address,bytes)

--NewAdmin(address,address)

--QuestionEmergencyResolved(bytes32,uint256[])

--QuestionFlagged(bytes32)

-- QuestionInitialized(bytes32,uint256,address,bytes,address,uint256,uint256)

-- QuestionPaused(bytes32)

-- QuestionReset(bytes32)

-- QuestionResolved(bytes32,int256,uint256[])

-- QuestionUnpaused(bytes32)

-- RemovedAdmin(address,address)



--SELECT *

--FROM ⁠ bigquery-public-data.crypto_polygon.logs ⁠

--WHERE address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74"

-- AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--LIMIT 1000;







-- gpt says token_transfer may be good, but i gnore for now









-- traces is low level eth



-- transactions - got lots of transations, may or may not have data. the functions dont have an obvious bet yes or no. Butttt, all contracts come from1 or 2 places. I query them!



--SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   to_address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74"

--   AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--LIMIT 1000



--, they are 0x39ba731cfa2828ea64787ae165fd1e78fe43aede and 0x91430cad2d3975766499717fa0d66a78d814e5c5





--SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   to_address = "0x91430cad2d3975766499717fa0d66a78d814e5c5"

--   AND block_timestamp BETWEEN "2024-09-01 00:00:00" AND "2024-11-25 23:59:59"

--LIMIT 1000





-- no data to eaiterh... H,mmm







--— next plan would be query transactions for my own key on a dat I traded!

-- the guru

-- 0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93



--SELECT nonce FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"

--   AND block_timestamp BETWEEN "2024-08-01 00:00:00" AND "2024-09-01 23:59:59"

--LIMIT 1000















-- we foun dthe guru!

-- SELECT to_address

-- FROM `bigquery-public-data.crypto_polygon.token_transfers`

-- WHERE from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"

--  AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"



-- SELECT DISTINCT to_address

-- FROM `bigquery-public-data.crypto_polygon.token_transfers`

-- WHERE from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"

--   AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"







  

  



-- SELECT * FROM `bigquery-public-data.crypto_polygon.transactions`

-- WHERE `hash` = "0x92a9c36cba557a469432328f6c59b05c5b487046465bf8bcf98e12b967caa9e7"

--    AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

-- LIMIT 1000





-- SELECT *

-- FROM `bigquery-public-data.crypto_polygon.logs`

-- WHERE transaction_hash = "0x92a9c36cba557a469432328f6c59b05c5b487046465bf8bcf98e12b967caa9e7"

-- AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

-- LIMIT 1000;









-- SELECT * FROM `bigquery-public-data.crypto_polygon.transactions`

-- WHERE to_address = "0x78769d50be1763ed1ca0d5e878d93f05aabff29e"

--    AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-02 23:59:59"





-- SELECT *

-- FROM `bigquery-public-data.crypto_polygon.logs`

-- where transaction_hash="0x48d4c13234d6b02f47a6580e452c7a3201e06378851c7b5eb2df42954789cb0c"

-- and

-- block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-02 23:59:59"

-- LIMIT 1000;







-- SELECT *

-- FROM `bigquery-public-data.crypto_polygon.logs`

-- WHERE transaction_hash IN (

--     SELECT `hash`

--     FROM `bigquery-public-data.crypto_polygon.transactions`

--     WHERE 

--     to_address = "0x78769d50be1763ed1ca0d5e878d93f05aabff29e"

--     AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--     Limit 

--     1000

-- ) and 

-- block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59";





SELECT *

FROM `bigquery-public-data.crypto_polygon.logs`

WHERE transaction_hash IN (

    SELECT `hash`

    FROM `bigquery-public-data.crypto_polygon.transactions`

    WHERE 

    to_address = "0x78769d50be1763ed1ca0d5e878d93f05aabff29e"

    AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

    Limit 

    1000

) and 

block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59";











































































--- Contracts



-- Goal to indeitfy polymarket contracts



-- tried  address for

-- 0x39Ba731cFA2828ea64787AE165FD1e78Fe43AEDe and 0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E 

-- hit on 0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74!

 --SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.contracts ⁠ where address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74" LIMIT 1000









--- Now I used that to attak the logs



-- I got loads of data but only about AncillaryDataUpdated(bytes32,address,bytes)

--NewAdmin(address,address)

--QuestionEmergencyResolved(bytes32,uint256[])

--QuestionFlagged(bytes32)

-- QuestionInitialized(bytes32,uint256,address,bytes,address,uint256,uint256)

-- QuestionPaused(bytes32)

-- QuestionReset(bytes32)

-- QuestionResolved(bytes32,int256,uint256[])

-- QuestionUnpaused(bytes32)

-- RemovedAdmin(address,address)



--SELECT *

--FROM ⁠ bigquery-public-data.crypto_polygon.logs ⁠

--WHERE address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74"

-- AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--LIMIT 1000;







-- gpt says token_transfer may be good, but i gnore for now









-- traces is low level eth



-- transactions - got lots of transations, may or may not have data. the functions dont have an obvious bet yes or no. Butttt, all contracts come from1 or 2 places. I query them!



--SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   to_address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74"

--   AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--LIMIT 1000



--, they are 0x39ba731cfa2828ea64787ae165fd1e78fe43aede and 0x91430cad2d3975766499717fa0d66a78d814e5c5





--SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   to_address = "0x91430cad2d3975766499717fa0d66a78d814e5c5"

--   AND block_timestamp BETWEEN "2024-09-01 00:00:00" AND "2024-11-25 23:59:59"

--LIMIT 1000





-- no data to eaiterh... H,mmm







--— next plan would be query transactions for my own key on a dat I traded!

-- the guru

-- 0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93



--SELECT nonce FROM ⁠ bigquery-public-data.crypto_polygon.transactions ⁠ 

--WHERE 

--   from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"

--   AND block_timestamp BETWEEN "2024-08-01 00:00:00" AND "2024-09-01 23:59:59"

--LIMIT 1000















-- we foun dthe guru!

-- SELECT *

-- FROM ⁠ bigquery-public-data.crypto_polygon.token_transfers ⁠

-- WHERE from_address = "0x9f47f1fcb1701bf9eaf31236ad39875e5d60af93"

--  AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

-- LIMIT 1000;





  

  



SELECT * FROM `bigquery-public-data.crypto_polygon.transactions`

WHERE `hash` = "0x92a9c36cba557a469432328f6c59b05c5b487046465bf8bcf98e12b967caa9e7"

   AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

LIMIT 1000Newest





--- Contracts



-- Goal to indeitfy polymarket contracts



-- tried  address for

-- 0x39Ba731cFA2828ea64787AE165FD1e78Fe43AEDe and 0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E 

-- hit on 0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74!

 --SELECT * FROM ⁠ bigquery-public-data.crypto_polygon.contracts ⁠ where address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74" LIMIT 1000









--- Now I used that to attak the logs



-- I got loads of data but only about AncillaryDataUpdated(bytes32,address,bytes)

--NewAdmin(address,address)

--QuestionEmergencyResolved(bytes32,uint256[])

--QuestionFlagged(bytes32)

-- QuestionInitialized(bytes32,uint256,address,bytes,address,uint256,uint256)

-- QuestionPaused(bytes32)

-- QuestionReset(bytes32)

-- QuestionResolved(bytes32,int256,uint256[])

-- QuestionUnpaused(bytes32)

-- RemovedAdmin(address,address)



--SELECT *

--FROM ⁠ bigquery-public-data.crypto_polygon.logs ⁠

--WHERE address = "0x6a9d222616c90fca5754cd1333cfd9b7fb6a4f74"

-- AND block_timestamp BETWEEN "2024-07-01 00:00:00" AND "2024-07-01 23:59:59"

--LIMIT 1000;







-- gpt says token_transfer may be good, but i gnore for now